In [113]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

In [114]:
ops.reset_default_graph()
sess = tf.Session()

In [246]:
def create_data(n_samples = 10, n_features=1):
    LOW = -100
    HIGH = 100
    SCALE = 1
    
    w = np.random.uniform(low=LOW, high=HIGH, size=(n_features, 1))
    print('weights: {}'.format(w.shape))
    
    X = np.random.uniform(low=LOW, high=HIGH, size=(n_samples, n_features))
    print('X: {}'.format(X.shape))
    
    b = np.random.rand() * (HIGH - LOW) + LOW
    print('b: {}'.format(b))
    
    noise = np.random.normal(loc=0, scale=SCALE, size=(n_samples, 1))
    print('noise: {}'.format(noise.shape))
    
    y = np.matmul(X, w) + b + noise
    print('y: {}'.format(y.shape))
    
    ones = np.ones(shape=(n_samples, 1))
    X1 = np.column_stack((X, ones))
    print('X1: {}'.format(X1.shape))
    
    return {
        'X': X1,
        'y': y,
        'w': w,
        'b': b
    }

In [247]:
def solve_matrix_inverse(X, y, sess):
    Xt = tf.transpose(X)
    return tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(Xt, X)), Xt), y)

In [248]:
def solve_decomposition(X, y, sess):
    Xt = tf.transpose(X)
    XtX = tf.matmul(Xt, X)
    L = tf.cholesky(XtX)
    return tf.matrix_solve(tf.transpose(L), tf.matrix_solve(L, tf.matmul(Xt, y)))

In [249]:
def solve_gradient_descent(X, y, sess):
    w_shape = [X.get_shape()[1].value, 1]
    w = tf.Variable(tf.random_normal(shape=w_shape, dtype=tf.float64))
    output = tf.matmul(X, w)
    loss = tf.reduce_mean(tf.square(output - y))
    optimizer = tf.train.AdamOptimizer(10)
    train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    
    ITERATIONS = 5000
    LOG_STEP = ITERATIONS // 5
    
    for i in range(ITERATIONS):
        sess.run(train)
#         if i == 0 or (i + 1) % LOG_STEP == 0:
#             loss_res = sess.run(loss)
#             print('#{} loss = {}'.format(i + 1, loss_res))
            
    return w

In [250]:
def solve(X_vals, y_vals, solve_fn):
    ops.reset_default_graph()
    sess = tf.Session()
    X = tf.constant(X_vals)
    y = tf.constant(y_vals)
    sol = solve_fn(X, y, sess)
    return sess.run(sol)

In [252]:
data = create_data(n_samples=3000, n_features=2000)

weights: (2000, 1)
X: (3000, 2000)
b: -1.4261481392171333
noise: (3000, 1)
y: (3000, 1)
X1: (3000, 2001)


In [253]:
X_vals = data['X']
print(X_vals[: 3, -4 :])

[[-92.17697219  -5.06948446  91.02355342   1.        ]
 [ 77.8050746   57.36591665  66.3750602    1.        ]
 [-27.80100168  52.04276302  -8.20161818   1.        ]]


In [254]:
y_vals = data['y']
print(y_vals[: 3])

[[-256267.86959633]
 [ -85810.429216  ]
 [ 202777.43003922]]


In [255]:
w = data['w']
w

array([[ 59.61560905],
       [ 24.33752084],
       [ 62.61961399],
       ..., 
       [ 92.11355498],
       [-98.43061662],
       [-92.45914371]])

In [256]:
b = data['b']
b

-1.4261481392171333

In [257]:
sol_mi = solve(X_vals, y_vals, solve_matrix_inverse)
sol_mi

array([[ 59.61598225],
       [ 24.33709839],
       [ 62.6204047 ],
       ..., 
       [-98.43057087],
       [-92.45929784],
       [ -1.48336288]])

In [258]:
sol_de = solve(X_vals, y_vals, solve_decomposition)
sol_de

array([[ 59.61598225],
       [ 24.33709839],
       [ 62.6204047 ],
       ..., 
       [-98.43057087],
       [-92.45929784],
       [ -1.48336288]])

In [259]:
sol_gd = solve(X_vals, y_vals, solve_gradient_descent)
sol_gd

array([[ 59.72382694],
       [ 24.30531389],
       [ 62.53698686],
       ..., 
       [-98.31336205],
       [-92.37317458],
       [ -1.45875258]])

In [260]:
%timeit solve(X_vals, y_vals, solve_matrix_inverse)

3.34 s ± 49.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [261]:
%timeit solve(X_vals, y_vals, solve_decomposition)

2.61 s ± 268 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [245]:
%timeit solve(X_vals, y_vals, solve_gradient_descent)

KeyboardInterrupt: 